In [1]:
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
# Credentials
DB_NAME = 'documentcoder'
DB_USER = 'postgres'
DB_PASS = 'save'

In [3]:
# CONNECTION IS ESTABLISHED IF NOTHING RETURNS 
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [4]:
# Create Cursor to execute statements
cur = conn.cursor()

In [5]:
# how many companies are in the dataset 
cur.execute('SELECT COUNT(*) FROM (SELECT DISTINCT id FROM api_policy) AS unique_policy_id_cnt;')
company_cnt = cur.fetchall()[0][0]
print(company_cnt)

628


In [6]:
# how many companies have its policies digested (aka policy instances)
cur.execute("""
            SELECT COUNT(*) 
            FROM (SELECT DISTINCT policy_id FROM api_policyinstance) AS unique_cnt;
            """)
policy_cnt = cur.fetchall()[0][0]
print(policy_cnt)

18


In [7]:
# how many coding instances are there
cur.execute("""
            SELECT id as coding_instance_id, COUNT(*)
            FROM (SELECT id, jsonb_object_keys(coding_values)
                  FROM api_codinginstance
                  WHERE coder_email != 'davidbstein@gmail.com') AS key_cnts
            GROUP BY key_cnts.id
            HAVING COUNT(*) != 1;
            """)
# the result is under the assumption that all codings 
# even a "bad one" ie has categoryHighlights in the coding_values object 
# => coding instances that have only a count of 1 is bad data 
coding_instance_cnt = len(cur.fetchall())
print(coding_instance_cnt)

25


In [8]:
# how many policy instances with two coding instances or more are there

# SELECT COUNT(*)
# FROM (SELECT policy_instance_id, count(DISTINCT coder_email) AS coder_cnt
#     FROM api_codinginstance 
#     WHERE coder_email != 'davidbstein@gmail.com'
#     GROUP BY policy_instance_id) AS sub
# WHERE sub.coder_cnt > 1;

cur.execute("""
            SELECT id, policy_instance_id, coding_values, coder_email
            FROM api_codinginstance
            WHERE coder_email != 'davidbstein@gmail.com'; 
            """)
query_res = cur.fetchall()
#print(query_res)

agg_dict = {}
for record in query_res: 
    encoding_len = len(record[2])
    if (encoding_len > 1):
        if record[1] not in agg_dict:
            agg_dict[record[1]] = 0
            
        agg_dict[record[1]] += 1
        
#print(agg_dict)
filtered_dict = {k:v for (k,v) in agg_dict.items() if v > 1}

mult_coding_inst_cnt = len(filtered_dict)
print(mult_coding_inst_cnt)

9


In [9]:
# how many policy instances with two coding instances or more AND corrections by FMW are there

# SELECT api_codinginstance.policy_instance_id, 
# COUNT(DISTINCT coder_email) as coder_cnt
# FROM api_codinginstance
# INNER JOIN
#     (SELECT policy_instance_id
#     FROM api_codinginstance
#     WHERE coder_email = 'florencia.m.wurgler@gmail.com') AS fmw_ids
# ON fmw_ids.policy_instance_id = api_codinginstance.policy_instance_id
# WHERE coder_email != 'davidbstein@gmail.com'
# GROUP BY api_codinginstance.policy_instance_id
# HAVING COUNT(DISTINCT coder_email) > 2;

cur.execute("""
            SELECT id, api_codinginstance.policy_instance_id, coder_email, coding_values
            FROM api_codinginstance
            INNER JOIN
                (SELECT policy_instance_id
                FROM api_codinginstance
                WHERE coder_email = 'florencia.m.wurgler@gmail.com') AS fmw_ids
            ON fmw_ids.policy_instance_id = api_codinginstance.policy_instance_id
            WHERE coder_email != 'davidbstein@gmail.com';
            """)

query_res = cur.fetchall()
#print(query_res)

agg_dict = {}
for record in query_res: 
    encoding_len = len(record[3])
    # checks for codings where theres actual content and not just categoryHighlights = {}
    if (encoding_len > 1):
        if record[1] not in agg_dict:
            agg_dict[record[1]] = 0
            
        agg_dict[record[1]] += 1
        
#print(agg_dict)
filtered_dict = {k:v for (k,v) in agg_dict.items() if v > 2}

mult_coding_inst_wFMW_cnt = len(filtered_dict)
print(mult_coding_inst_wFMW_cnt)

1


In [10]:
cur.execute("""
            SELECT x->'questions'
            FROM api_coding, jsonb_array_elements(categories) as x""")

query_res = cur.fetchall()
#print(query_res)

q_ids = []

for record in query_res: 
    for q in record[0]:
        q_ids.append(q['id'])

print(q_ids)

['v72.1_2020.1', 'v72.2_2020.1', 'v73_2020.1', 'v74_2020.1', 'v75_2020.1', 'v76_2020.1', 'v77_2020', 'v78_2020.1', 'v72_2020.1', 'v80_2020.1', 'v3_2020', 'v10.1_2020', 'v10.2_2020.1', 'v11.1_2020', 'v12_2020.1', 'v13_2020.1', 'v17_2020', 'v18_2020', 'v19_2020', 'v32_2020', 'v35_2020', 'v36_2020', 'v48_2020', 'v20_2020.1', 'v21_2020.1', 'v22_2020.1', 'v27_2020.1', 'v28_2020', 'v29_2020', 'v31_2020', 'v37_2020', 'v38_2020.1', 'v37.2_2020.1', 'v39_2020', 'v40_2020.1', 'v47_2020.1', 'v49_2020', 'v51_2020', 'v41_2020', 'v42_2020', 'v43_2020', 'v44_2020', 'v45_2020', 'v46_2020', 'v53_2020', 'v54_2020', 'v52_2020', 'v55_2020', 'v56_2020', 'v57_2020', 'v58_2020', 'v59_2020', 'v60_2020', 'v61_2020', 'v62_2020.1', 'v63_2020', 'v64_2020', 'PP_in_TOU', 'v71_2020.1', 'v80.2_2020.1', 'v80.3_2020.1', 'v81_2020', 'v82_2020', 'v83_2020', 'v2_2020_txt', 'v0_2020.1', 'v1_2020', 'v2.1_2020', 'v2.2_2020', 'v2.3_2020.1', 'v2.4_2020.1']


In [40]:
# creating policy based stats 
cur.execute("""
            SELECT id, api_codinginstance.policy_instance_id, coder_email, coding_values
            FROM api_codinginstance
            WHERE coder_email != 'davidbstein@gmail.com';
            """)

query_res = cur.fetchall()
policy_instances = {}

for record in query_res: 
    incomplete_flag = False
    coded_q_ids = record[3].keys()
    #print(coded_q_ids)
        
    for q_id in q_ids:
        if q_id not in coded_q_ids:
            #print('incomplete')
            incomplete_flag = True
    #print('-------------')
        
    temp = {'coding_instance_id': record[0],
            'coder_email': record[2],
            'num_encodings': len(record[3]),
            'complete_encoding': False if incomplete_flag else True}
    
    if record[1] not in policy_instances:
        policy_instances[record[1]] = {'codings': [], 'total_num_encodings': 0, 'num_completed_encodings': 0}
    
    policy_instances[record[1]]['codings'].append(temp)


total_num = []
num_complete = []
# policy_instance_id = key here
for key in policy_instances: 
    policy_instances[key]['total_num_encodings'] = len(policy_instances[key]['codings'])
    total_num.append(policy_instances[key]['total_num_encodings'])
    
    cnt = 0
    for coding in policy_instances[key]['codings']:
        if coding['complete_encoding'] == True:
            cnt += 1
    
    policy_instances[key]['num_completed_encodings'] = cnt    
    num_complete.append(policy_instances[key]['num_completed_encodings'])

print(policy_instances)

{1: {'codings': [{'coding_instance_id': 8, 'coder_email': 'gfc9001@nyu.edu', 'num_encodings': 66, 'complete_encoding': False}, {'coding_instance_id': 10, 'coder_email': 'kat9234@nyu.edu', 'num_encodings': 65, 'complete_encoding': False}, {'coding_instance_id': 11, 'coder_email': 'mc8562@nyu.edu', 'num_encodings': 35, 'complete_encoding': False}, {'coding_instance_id': 13, 'coder_email': 'florencia.m.wurgler@gmail.com', 'num_encodings': 62, 'complete_encoding': False}, {'coding_instance_id': 15, 'coder_email': 'dbs438@nyu.edu', 'num_encodings': 7, 'complete_encoding': False}], 'total_num_encodings': 5, 'num_completed_encodings': 0}, 10: {'codings': [{'coding_instance_id': 17, 'coder_email': 'mc8562@nyu.edu', 'num_encodings': 65, 'complete_encoding': False}, {'coding_instance_id': 18, 'coder_email': 'gfc9001@nyu.edu', 'num_encodings': 65, 'complete_encoding': False}], 'total_num_encodings': 2, 'num_completed_encodings': 0}, 12: {'codings': [{'coding_instance_id': 19, 'coder_email': 'gfc9

In [49]:
df = pd.DataFrame(data={'policy_instance_id': policy_instances.keys(), 'total_num_encodings': total_num, 
                        'num_completed_encodings': num_complete})

In [51]:
df

,policy_instance_id,total_num_encodings,num_completed_encodings
0,1,5,0
1,10,2,0
2,12,3,0
3,11,3,0
4,5,3,0
5,3,3,0
6,8,3,0
7,13,2,0
8,14,1,0
9,4,1,0
